# Gold Orchestrator Notebook
This notebook executes the Gold Layer pipeline: dimensions, facts, KPIs, and validations.

**Author:** Diego Mayorga  
**Compute:** Serverless (Unity Catalog enforced)


In [ ]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.getOrCreate()

In [ ]:
# -----------------------------------------------------------------------------
# WIDGETS (DEFAULT VALUES FILLED)
# -----------------------------------------------------------------------------
dbutils.widgets.text("run_pipeline", "yes")
dbutils.widgets.text("stop_on_error", "yes")
dbutils.widgets.text("run_validation", "yes")
dbutils.widgets.text("timeout_seconds", "6000")

dbutils.widgets.text("dim_date_path", "/Workspace/Users/diego.mayorgacapera@gmail.com/.bundle/BI_Market_Visibility/dev/files/notebooks/gold/dimensions/gold_dim_date")
dbutils.widgets.text("dim_product_path", "/Workspace/Users/diego.mayorgacapera@gmail.com/.bundle/BI_Market_Visibility/dev/files/notebooks/gold/dimensions/gold_dim_product")
dbutils.widgets.text("dim_pdv_path", "/Workspace/Users/diego.mayorgacapera@gmail.com/.bundle/BI_Market_Visibility/dev/files/notebooks/gold/dimensions/gold_dim_pdv")

In [ ]:
# Read widgets
RUN_PIPELINE = dbutils.widgets.get("run_pipeline") == "yes"
STOP_ON_ERROR = dbutils.widgets.get("stop_on_error") == "yes"
RUN_VALIDATION = dbutils.widgets.get("run_validation") == "yes"
TIMEOUT = int(dbutils.widgets.get("timeout_seconds"))

In [ ]:
# -----------------------------------------------------------------------------
# UNITY CATALOG CONFIG (MANDATORY)
# -----------------------------------------------------------------------------
CATALOG = "workspace"
SCHEMA_GOLD = "default"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA_GOLD}")

In [ ]:
# -----------------------------------------------------------------------------
# NOTEBOOK PATHS VALIDATION
# -----------------------------------------------------------------------------
def validate_notebook_path(path, name):
    if not path or path.strip() == "":
        raise ValueError(f"❌ Path for {name} is empty. Check pipeline YAML and parameters.")
    if path.endswith("/gold") or path.endswith("/gold/"):
        raise ValueError(f"❌ Path for {name} points to a folder, not a notebook: {path}")
    if "/gold/dimensions/" not in path:
        raise ValueError(f"❌ Path for {name} does not point to a valid dimensions notebook: {path}")
    return path

NOTEBOOKS = {
    "dim_date": validate_notebook_path(dbutils.widgets.get("dim_date_path"), "dim_date"),
    "dim_product": validate_notebook_path(dbutils.widgets.get("dim_product_path"), "dim_product"),
    "dim_pdv": validate_notebook_path(dbutils.widgets.get("dim_pdv_path"), "dim_pdv"),
    # facts and kpis can be added later
}

In [ ]:
# -----------------------------------------------------------------------------
# EXECUTION ENGINE
# -----------------------------------------------------------------------------
def run_notebook(name: str, path: str):
    print(f"\n▶ Executing: {name}")
    start = datetime.now()

    try:
        result = dbutils.notebook.run(
            path,
            timeout_seconds=TIMEOUT,
            arguments={}
        )
        duration = (datetime.now() - start).seconds
        print(f"✔ {name} completed in {duration}s")
        return result

    except Exception as e:
        print(f"✖ ERROR in {name}")
        print(str(e))
        if STOP_ON_ERROR:
            raise
        else:
            return None

In [ ]:
# -----------------------------------------------------------------------------
# PIPELINE EXECUTION
# -----------------------------------------------------------------------------
if RUN_PIPELINE:
    print("🚀 Starting GOLD pipeline")

    for name, path in NOTEBOOKS.items():
        run_notebook(name, path)

    print("✅ GOLD pipeline finished")

else:
    print("⏭ Pipeline execution skipped by widget")

In [ ]:
# -----------------------------------------------------------------------------
# OPTIONAL VALIDATION
# -----------------------------------------------------------------------------
if RUN_VALIDATION:
    print("\n🔍 Validation phase (placeholder)")
    # Here you can add Delta checks / counts / constraints